LLM Prompt

In [ ]:
# System prompt describes information given to all conversations
system_prompt = """
You are a helpful, respectful and honest assistant for labeling topics.
"""

main_prompt = """
I have a topic that contains the following documents:
[DOCUMENTS]
The topic is described by the following keywords: [KEYWORDS]

Based on the information above, extract a short topic label in the following format:
topic: <topic label>
"""


prompt = system_prompt  + main_prompt


BERTopic + LLM

In [ ]:
from sklearn.metrics import silhouette_score
from bertopic import BERTopic
from umap import UMAP
import numpy as np
from hdbscan import HDBSCAN
from bertopic.vectorizers import ClassTfidfTransformer
from sklearn.cluster import KMeans
from sentence_transformers import SentenceTransformer
from sklearn.feature_extraction.text import CountVectorizer
import openai
import tiktoken
from bertopic.representation import OpenAI
from bertopic import BERTopic
import nltk
import gensim
import gensim.corpora as corpora
import pandas as pd
import numpy as np
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer
from umap import UMAP
from hdbscan import HDBSCAN
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer
from bertopic.vectorizers import ClassTfidfTransformer
from gensim.models.coherencemodel import CoherenceModel
import pandas as pd
import re

import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import fetch_20newsgroups

from gensim.models.coherencemodel import CoherenceModel
from gensim.corpora.dictionary import Dictionary

In [ ]:
import openai
from bertopic.representation import OpenAI
from bertopic import BERTopic

silhouette_score_list = []
sentence_model = SentenceTransformer("all-MiniLM-L6-v2")
# cluster_model = KMeans(31)
embeddings = sentence_model.encode(df1['Future Work'], show_progress_bar=False)
# Tokenizer
tokenizer= tiktoken.encoding_for_model("gpt-4o-mini")
# others: gpt-4-turbo-preview	, gpt-4, 	gpt-4-0613	, gpt-4-32k	, gpt-4-32k-0613

# umap_model = UMAP(n_neighbors=13, n_components=15,min_dist=0.0, metric='cosine',random_state=42)

vectorizer_model = CountVectorizer(stop_words="english")

# Create your representation model
client = openai.OpenAI(api_key="")

representation_model = OpenAI(
    client,
    model="gpt-4o-mini",
    delay_in_seconds=2,
    chat=True,
    nr_docs=4,
    doc_length=100,
    tokenizer=tokenizer,
    prompt = prompt
)

topic_model = BERTopic(
    embedding_model="thenlper/gte-small",
    zeroshot_min_similarity=.75,
    representation_model=representation_model,
    vectorizer_model=vectorizer_model
)

topics, probs = topic_model.fit_transform(df1['Future Work'], embeddings)

# Generate `X` and `labels` only for non-outlier topics (as they are technically not clusters)
umap_embeddings = topic_model.umap_model.transform(embeddings)
indices = [index for index, topic in enumerate(topics) if topic != -1]
X = umap_embeddings[np.array(indices)]
labels = [topic for index, topic in enumerate(topics) if topic != -1]

  # Calculate silhouette score
print(silhouette_score(X, labels))
most_representative_words_llm = topic_model.get_topic_info()

from keybert import KeyBERT

kw_model = KeyBERT()
doc_embeddings, word_embeddings = kw_model.extract_embeddings(df1["Future Work"])
keywords = kw_model.extract_keywords(df1["Future Work"], doc_embeddings=doc_embeddings, word_embeddings=word_embeddings)
print(keywords)
most_representative_words_llm.to_csv("most_representative_words_llm1.csv",index=False)

measuring coherence score

In [ ]:
words_only = [[word for word, _ in sublist] for sublist in keywords]
# convert list to string
df1["Future Work"] = df1["Future Work"].apply(lambda x: ''.join(x))
# Tokenize the strings to create a list of lists
list_of_tokens = df1["Future Work"].apply(lambda x: x.split()).tolist()

word2id = Dictionary( list_of_tokens )

# Coherence model
cm = CoherenceModel(topics= words_only,
                    texts=list_of_tokens,
                    coherence='c_v',
                    dictionary=word2id)

coherence_per_topic = cm.get_coherence_per_topic()
print(coherence_per_topic)

import statistics
print(statistics.mean(coherence_per_topic))

visualization

In [ ]:
topic_model.visualize_heatmap()

In [ ]:
from umap import UMAP

# Run the visualization with the original embeddings
topic_model.visualize_documents(df1['Future Work'], embeddings=embeddings)

# Reduce dimensionality of embeddings, this step is optional but much faster to perform iteratively:
reduced_embeddings = UMAP(n_neighbors=10, n_components=2, min_dist=0.0, metric='cosine').fit_transform(embeddings)
topic_model.visualize_documents(df1['Future Work'], reduced_embeddings=reduced_embeddings)

In [ ]:
from bertopic import BERTopic

# topic_model = BERTopic()
# topics, _ = topic_model.fit_transform(docs)
topic_distr, _ = topic_model.approximate_distribution(df1['Future Work'], min_similarity=0)

In [ ]:
# don't use embedding_model = sentence model in 'BERTopic'
topic_model = BERTopic(representation_model=representation_model,vectorizer_model=vectorizer_model,
                          verbose=True, calculate_probabilities=True)

topics, probs = topic_model.fit_transform(df1['Future Work'], embeddings)

In [ ]:
# To visualize the probabilities of topic assignment
topic_model.visualize_distribution(probs[0])

# To visualize the topic distributions in a document
topic_model.visualize_distribution(topic_distr[0])
